In [1]:
%pip install pandas
%pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import numpy as np
import shutil
from sklearn.cluster import KMeans
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras._tf_keras.keras.preprocessing import image
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from keras._tf_keras.keras.applications import MobileNetV2, EfficientNetB0
# Other to try or combine: ResNet50, Xception, VGG16, InceptionV3
from keras._tf_keras.keras.models import Model, Sequential, load_model
from keras._tf_keras.keras.utils import load_img, img_to_array, to_categorical 
from keras._tf_keras.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras._tf_keras.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import warnings

In [11]:
source_file = "source_directory/WARDE_WEB.csv"
destination_file = "source_directory/Image_Tagging_Dataset.csv"
image_folder = "source_directory/Warde"
new_image_folder = "source_directory/Images"

CATEGORIES = {
    "TYPE": [
        "velvet", "curtain", "double purpose", "upholstery", "wallcovering",
        "embroidery", "faux fur", "faux leather", "jacquard", "microfiber",
        "organic", "print & embossed", "satin", "sheer", "suede", "sunscreen",
        "wallpanel", "wallpaper", "weave"
    ],
    "COLOR": [
        "black", "blue", "brown", "dark beige", "dark grey", "green", "light beige",
        "light grey", "metallic", "multicolor", "orange", "pink", "purple", "red",
        "white", "yellow"
    ],
    "STYLE": [
        "children", "classical", "contemporary & modern", "ethnic & oriental", "floral",
        "geometric", "illustrative", "stripes; checks; and zigzags", "plain", "textured"
    ],
    "USAGE": [
        "curtain", "double purpose", "upholstery", "wallcovering"
    ]
}

In [12]:
if os.path.exists(new_image_folder) or os.path.exists(destination_file):
    print("The folder or CSV file already exists.")
    raise SystemExit()

os.makedirs(new_image_folder, exist_ok=True)

columns_to_keep = [
    "Item Code", "Usage", "TYPE 1", "TYPE 2", "TYPE 3", 
    "Color1", "Color2", "Color3", "Style 1", "Style 2", "Style 3"
]

df = pd.read_csv(source_file)
df['Item Code'] = df['Item Code'].apply(lambda x: (x.strip().replace(" ", "") + ".jpg"))
filtered_df = df[columns_to_keep]
columns_to_check = ["TYPE 2", "TYPE 3", "Color2", "Color3", "Style 2", "Style 3"]
filtered_df[columns_to_check] = filtered_df[columns_to_check].replace("0", np.nan)
filtered_df.loc[:, filtered_df.columns != "Item Code"] = (
    filtered_df.loc[:, filtered_df.columns != "Item Code"]
    .applymap(lambda x: x.lower() if isinstance(x, str) else x)
)

def image_exists(item_code):

    # if item_code == "MNM6009.jpg":
    #     return False
    # Not nessescarly this file, but 
    # Copying large images sometimes 
    # cause errors

    image_file = os.path.join(image_folder, f"{item_code}")
    image_file = image_file.replace("\\", "/")
    if os.path.exists(image_file):
        destination_image_path = os.path.join(new_image_folder, os.path.basename(image_file))
        shutil.copy(image_file, destination_image_path)
    return os.path.exists(image_file)

filtered_df = filtered_df[filtered_df["Item Code"].apply(image_exists)]
filtered_df.to_csv(destination_file, index=False)

print(f"Filtered dataset and Images saved to {destination_file} and {new_image_folder}")

Filtered dataset and Images saved to source_directory/Image_Tagging_Dataset.csv and source_directory/Images


In [13]:
warnings.filterwarnings("ignore")
data = pd.read_csv(destination_file)

def binary_encode(df, column_prefix, categories):
    encoded = pd.DataFrame(0, index=df.index, columns=categories)
    for col in [col for col in df.columns if column_prefix in col]:
        for category in categories:
            encoded[category] |= (df[col] == category).astype(int)
    return encoded

type_encoded = binary_encode(data, "TYPE", CATEGORIES["TYPE"])
color_encoded = binary_encode(data, "Color", CATEGORIES["COLOR"])
style_encoded = binary_encode(data, "Style", CATEGORIES["STYLE"])
usage_encoded = pd.get_dummies(data["Usage"], prefix="Usage")
usage_encoded = usage_encoded.reindex(
    columns=[f"Usage_{cat}" for cat in CATEGORIES["USAGE"]],
    fill_value=0
)

encoded_labels = pd.concat([usage_encoded, type_encoded, color_encoded, style_encoded, ], axis=1)
encoded_labels = encoded_labels.astype("float32")

image_directory = "source_directory/Images"  
data["source_directory/Images"] = data["Item Code"].apply(lambda x: os.path.join(image_directory, x))
data = data[data["source_directory/Images"].apply(os.path.exists)]

def load_images(file_paths, target_size=(224, 224)):
    images = []
    for destination_file in file_paths:
        img = tf.keras.utils.load_img(destination_file, target_size=target_size)
        img = tf.keras.utils.img_to_array(img) / 255.0  
        images.append(img)
    return np.array(images)

images = load_images(data["source_directory/Images"])
labels = encoded_labels.loc[data.index].values


In [14]:
print(f"USAGE (expected 4): {usage_encoded.shape[1]}")
print(f"TYPE (expected 20): {type_encoded.shape[1]}")
print(f"COLOR (expected 17): {color_encoded.shape[1]}")
print(f"STYLE (expected 11): {style_encoded.shape[1]}")

# USAGE (expected 4): 4
# TYPE (expected 20): 19
# COLOR (expected 17): 16
# STYLE (expected 11): 10

# Values check after encoding

USAGE (expected 4): 4
TYPE (expected 20): 19
COLOR (expected 17): 16
STYLE (expected 11): 10


In [15]:
# Without Pretrained Model

X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

usage_train = y_train[:, :4]
type_train = y_train[:, 4:24]
color_train = y_train[:, 24:41]
style_train = y_train[:, 41:]

usage_val = y_val[:, :4]
type_val = y_val[:, 4:24]
color_val = y_val[:, 24:41]
style_val = y_val[:, 41:]

usage_test = y_test[:, :4]
type_test = y_test[:, 4:24]
color_test = y_test[:, 24:41]
style_test = y_test[:, 41:]

y_train_type_color_style = np.concatenate([type_train, color_train, style_train], axis=1)
y_val_type_color_style = np.concatenate([type_val, color_val, style_val], axis=1)
y_test_type_color_style = np.concatenate([type_test, color_test, style_test], axis=1)

def custom_data_gen(x, y_type_color_style, y_usage, batch_size):
    num_samples = x.shape[0]
    while True:
        for offset in range(0, num_samples, batch_size):
            x_batch = x[offset:offset+batch_size]
            y_type_color_style_batch = y_type_color_style[offset:offset+batch_size]
            y_usage_batch = y_usage[offset:offset+batch_size]
            yield x_batch, {'type_color_style': y_type_color_style_batch, 'usage': y_usage_batch}

batch_size = 32
train_data_gen = custom_data_gen(X_train, y_train_type_color_style, usage_train, batch_size)
val_data_gen = custom_data_gen(X_val, y_val_type_color_style, usage_val, batch_size)

input_layer = Input(shape=(224, 224, 3))
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)

type_color_style_output = Dense(y_train_type_color_style.shape[1], activation='sigmoid', name='type_color_style')(x)
usage_output = Dense(4, activation='softmax', name='usage')(x)

model = Model(inputs=input_layer, outputs=[type_color_style_output, usage_output])
model.compile(
    optimizer='adam',
    loss={'type_color_style': 'binary_crossentropy', 'usage': 'categorical_crossentropy'},
    metrics={'type_color_style': 'accuracy', 'usage': 'accuracy'}
)

train_steps_per_epoch = np.ceil(len(X_train) / batch_size).astype(int)
val_steps_per_epoch = np.ceil(len(X_val) / batch_size).astype(int)

history = model.fit(
    train_data_gen,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=val_data_gen,
    validation_steps=val_steps_per_epoch,
    epochs=10
)

model.save("multi_label_model.keras")

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 202s 9s/step - loss: 1.1843 - type_color_style_accuracy: 0.2177 - type_color_style_loss: 0.5709 - usage_accuracy: 0.8416 - usage_loss: 0.6123 - val_loss: 0.2724 - val_type_color_style_accuracy: 0.9630 - val_type_color_style_loss: 0.2709 - val_usage_accuracy: 1.0000 - val_usage_loss: 6.1467e-08
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 135s 9s/step - loss: 0.2899 - type_color_style_accuracy: 0.9753 - type_color_style_loss: 0.2898 - usage_accuracy: 1.0000 - usage_loss: 1.1900e-07 - val_loss: 0.2083 - val_type_color_style_accuracy: 0.9630 - val_type_color_style_loss: 0.2062 - val_usage_accuracy: 1.0000 - val_usage_loss: 8.6914e-06
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 165s 10s/step - loss: 0.2326 - type_color_style_accuracy: 0.9753 - type_color_style_loss: 0.2325 - usage_accuracy: 1.0000 - usage_loss: 1.7150e-05 - val_loss: 0.2047 - val_type_color_style_accuracy: 0.9630 - val_type_color_style_loss: 0.2026 - val_usage_accuracy: 1.0000 - val_usage_loss: 1.5

In [21]:
# MobileNetV2

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

usage_train = y_train[:, :4]
type_train = y_train[:, 4:23]
color_train = y_train[:, 23:39]
style_train = y_train[:, 39:]

usage_val = y_val[:, :4]
type_val = y_val[:, 4:23]
color_val = y_val[:, 23:39]
style_val = y_val[:, 39:]

usage_test = y_test[:, :4]
type_test = y_test[:, 4:23]
color_test = y_test[:, 23:39]
style_test = y_test[:, 39:]

y_train_type_color_style = np.concatenate([type_train, color_train, style_train], axis=1)

input_layer = Input(shape=(224, 224, 3))
x = base_model(input_layer)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)

type_color_style_output = Dense(y_train_type_color_style.shape[1], activation='sigmoid', name='type_color_style')(x)
usage_output = Dense(4, activation='softmax', name='usage')(x)

model = Model(inputs=input_layer, outputs=[type_color_style_output, usage_output])
model.compile(
    optimizer='adam',
    loss={'type_color_style': 'binary_crossentropy', 'usage': 'categorical_crossentropy'},
    metrics={'type_color_style': 'accuracy', 'usage': 'accuracy'}
)

train_steps_per_epoch = np.ceil(len(X_train) / batch_size).astype(int)
val_steps_per_epoch = np.ceil(len(X_val) / batch_size).astype(int)

train_data_gen = custom_data_gen(X_train, np.concatenate([type_train, color_train, style_train], axis=1), usage_train, batch_size)
val_data_gen = custom_data_gen(X_val, np.concatenate([type_val, color_val, style_val], axis=1), usage_val, batch_size)

history = model.fit(
    train_data_gen,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=val_data_gen,
    validation_steps=val_steps_per_epoch,
    epochs=10
)

model.save("multi_label_mobilenetv2_model.keras")


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 121s 4s/step - loss: 1.1449 - type_color_style_accuracy: 0.3589 - type_color_style_loss: 0.5809 - usage_accuracy: 0.7973 - usage_loss: 0.5631 - val_loss: 0.2078 - val_type_color_style_accuracy: 0.9630 - val_type_color_style_loss: 0.2050 - val_usage_accuracy: 1.0000 - val_usage_loss: 0.0000e+00
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - loss: 0.2230 - type_color_style_accuracy: 0.9713 - type_color_style_loss: 0.2230 - usage_accuracy: 1.0000 - usage_loss: 0.0000e+00 - val_loss: 0.1635 - val_type_color_style_accuracy: 0.9630 - val_type_color_style_loss: 0.1613 - val_usage_accuracy: 1.0000 - val_usage_loss: 0.0000e+00
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 0.1903 - type_color_style_accuracy: 0.9050 - type_color_style_loss: 0.1902 - usage_accuracy: 1.0000 - usage_loss: 0.0000e+00 - val_loss: 0.1543 - val_type_color_style_accuracy: 0.9259 - val_type_color_style_loss: 0.1537 - val_usage_accuracy: 1.0000 - val_usage_loss: 0.0000

In [23]:
# EfficientNetB0

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

usage_train = y_train[:, :4]
type_train = y_train[:, 4:24]
color_train = y_train[:, 24:41]
style_train = y_train[:, 41:]

usage_val = y_val[:, :4]
type_val = y_val[:, 4:24]
color_val = y_val[:, 24:41]
style_val = y_val[:, 41:]

usage_test = y_test[:, :4]
type_test = y_test[:, 4:24]
color_test = y_test[:, 24:41]
style_test = y_test[:, 41:]

input_layer = Input(shape=(224, 224, 3))
x = base_model(input_layer)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)

y_train_type_color_style = np.concatenate([type_train, color_train, style_train], axis=1)

type_color_style_output = Dense(y_train_type_color_style.shape[1], activation='sigmoid', name='type_color_style')(x)
usage_output = Dense(4, activation='softmax', name='usage')(x)

model = Model(inputs=input_layer, outputs=[type_color_style_output, usage_output])
model.compile(
    optimizer='adam',
    loss={'type_color_style': 'binary_crossentropy', 'usage': 'categorical_crossentropy'},
    metrics={'type_color_style': 'accuracy', 'usage': 'accuracy'}
)

train_steps_per_epoch = np.ceil(len(X_train) / batch_size).astype(int)
val_steps_per_epoch = np.ceil(len(X_val) / batch_size).astype(int)

train_data_gen = custom_data_gen(X_train, y_train_type_color_style, usage_train, batch_size)
val_data_gen = custom_data_gen(X_val, y_val_type_color_style, usage_val, batch_size)

history = model.fit(
    train_data_gen,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=val_data_gen,
    validation_steps=val_steps_per_epoch,
    epochs=10
)

model.save("multi_label_model_effnetb0.keras")

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 322s 9s/step - loss: 0.8918 - type_color_style_accuracy: 0.4384 - type_color_style_loss: 0.5429 - usage_accuracy: 0.8100 - usage_loss: 0.3482 - val_loss: 0.2119 - val_type_color_style_accuracy: 0.9630 - val_type_color_style_loss: 0.2108 - val_usage_accuracy: 1.0000 - val_usage_loss: 1.2014e-07
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 86s 5s/step - loss: 0.2286 - type_color_style_accuracy: 0.9753 - type_color_style_loss: 0.2287 - usage_accuracy: 1.0000 - usage_loss: 3.8822e-07 - val_loss: 0.1995 - val_type_color_style_accuracy: 0.9630 - val_type_color_style_loss: 0.1973 - val_usage_accuracy: 1.0000 - val_usage_loss: 0.0000e+00
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - loss: 0.2249 - type_color_style_accuracy: 0.9753 - type_color_style_loss: 0.2249 - usage_accuracy: 1.0000 - usage_loss: 2.3389e-08 - val_loss: 0.1965 - val_type_color_style_accuracy: 0.9630 - val_type_color_style_loss: 0.1947 - val_usage_accuracy: 1.0000 - val_usage_loss: 0.0000

In [16]:
def extract_dominant_colors(image_path, num_colors=3):
    img = Image.open(image_path)
    img = img.convert("RGB")
    pixels = np.array(img)
    
    pixels = pixels.reshape((-1, 3))
    kmeans = KMeans(n_clusters=num_colors).fit(pixels)
    
    dominant_colors = kmeans.cluster_centers_.astype(int)
    
    return [tuple(color) for color in dominant_colors]

def get_closest_colors(dominant_rgb_colors, color_mapping, max_matches=3):
    matched_colors = set()
    
    for rgb in dominant_rgb_colors:
        closest_color = None
        min_distance = float('inf')
        
        for color_name, color_rgb in color_mapping.items():
            distance = np.linalg.norm(np.array(rgb) - np.array(color_rgb))
            if distance < min_distance:
                closest_color = color_name
                min_distance = distance
        
        if closest_color and closest_color not in matched_colors:
            matched_colors.add(closest_color)
            if len(matched_colors) >= max_matches:
                break
    
    return list(matched_colors)

def predict_colors(image_path, color_mapping, num_colors=3):
    dominant_colors = extract_dominant_colors(image_path, num_colors)
    matched_colors = get_closest_colors(dominant_colors, color_mapping)
    return matched_colors

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict_single_image(model, image_path, thresholds=None):
    if thresholds is None:
        thresholds = {
            "TYPE": 0.5,
            "COLOR": 0.5,
            "STYLE": 0.5,
            "USAGE": 0.5
        }

    img_array = preprocess_image(image_path)
    type_color_style_pred, usage_pred = model.predict(img_array)
    type_color_style_binary = (type_color_style_pred > thresholds["TYPE"]).astype(float)
    usage_binary = np.argmax(usage_pred, axis=1)

    return type_color_style_pred, usage_pred, type_color_style_binary, usage_binary

def decode_predictions_with_probabilities(type_color_style_pred, usage_pred, thresholds):
    decoded = {}

    usage_idx = np.argmax(usage_pred, axis=1)
    decoded["USAGE"] = [(CATEGORIES["USAGE"][usage_idx[0]], float(np.max(usage_pred)))]

    num_type_labels = len(CATEGORIES["TYPE"])
    type_probabilities = type_color_style_pred[0, :num_type_labels]
    type_binary = (type_probabilities > thresholds["TYPE"]).astype(int)
    decoded["TYPE"] = sorted(
        [(CATEGORIES["TYPE"][i], float(type_probabilities[i])) for i, value in enumerate(type_binary) if value == 1],
        key=lambda x: x[1], 
        reverse=True
    )

    num_style_labels = len(CATEGORIES["STYLE"])
    style_probabilities = type_color_style_pred[0, num_type_labels:num_type_labels+num_style_labels]
    style_binary = (style_probabilities > thresholds["STYLE"]).astype(int)
    decoded["STYLE"] = sorted(
        [(CATEGORIES["STYLE"][i], float(style_probabilities[i])) for i, value in enumerate(style_binary) if value == 1],
        key=lambda x: x[1],  
        reverse=True
    )

    num_color_labels = len(CATEGORIES["COLOR"])
    color_probabilities = type_color_style_pred[0, num_type_labels+num_style_labels:num_type_labels+num_style_labels+num_color_labels]
    color_binary = (color_probabilities > thresholds["COLOR"]).astype(int)
    decoded["COLOR"] = sorted(
        [(CATEGORIES["COLOR"][i], float(color_probabilities[i])) for i, value in enumerate(color_binary) if value == 1],
        key=lambda x: x[1],
        reverse=True
    )

    return decoded


In [17]:
warnings.filterwarnings("ignore")

test_loss, type_color_style_loss, usage_loss, type_color_style_accuracy, usage_accuracy = model.evaluate(
    X_test, {'type_color_style': y_test_type_color_style, 'usage': usage_test}
)

print(f"Test Loss: {test_loss}")
print(f"Type-Color-Style Loss: {type_color_style_loss}, Type-Color-Style Accuracy: {type_color_style_accuracy}")
print(f"Usage Loss: {usage_loss}, Usage Accuracy: {usage_accuracy}")

predictions = model.predict(X_test)

type_color_style_predictions = predictions[0]
usage_predictions = predictions[1]

threshold = 0.2
binary_type_color_style_predictions = (type_color_style_predictions > threshold).astype(float)

binary_type_color_style_flat = binary_type_color_style_predictions.flatten()
usage_flat = usage_predictions.flatten()

accuracy = accuracy_score(y_test_type_color_style.flatten(), binary_type_color_style_flat)
precision = precision_score(y_test_type_color_style.flatten(), binary_type_color_style_flat, average="macro")
recall = recall_score(y_test_type_color_style.flatten(), binary_type_color_style_flat, average="macro")
f1 = f1_score(y_test_type_color_style.flatten(), binary_type_color_style_flat, average="macro")

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

print("Classification Report (Type-Color-Style):")
print(classification_report(y_test_type_color_style.flatten(), binary_type_color_style_flat))

usage_predictions_binary = np.argmax(usage_predictions, axis=1)
usage_test_binary = np.argmax(usage_test, axis=1)

usage_accuracy = accuracy_score(usage_test_binary, usage_predictions_binary)
usage_precision = precision_score(usage_test_binary, usage_predictions_binary, average="macro")
usage_recall = recall_score(usage_test_binary, usage_predictions_binary, average="macro")
usage_f1 = f1_score(usage_test_binary, usage_predictions_binary, average="macro")

print(f"Usage Accuracy: {usage_accuracy}")
print(f"Usage Precision: {usage_precision}")
print(f"Usage Recall: {usage_recall}")
print(f"Usage F1 Score: {usage_f1}")

usage_test_indices = np.argmax(usage_test, axis=1)  
usage_predictions_indices = np.argmax(usage_predictions, axis=1)  

print("Classification Report (Usage):")
print(classification_report(
    usage_test_indices,
    usage_predictions_indices,
    target_names=CATEGORIES["USAGE"],
    labels=list(range(len(CATEGORIES["USAGE"]))) 
))
    

4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 0.2106 - type_color_style_accuracy: 0.9701 - type_color_style_loss: 0.2079 - usage_accuracy: 1.0000 - usage_loss: 7.7138e-04
Test Loss: 0.20814372599124908
Type-Color-Style Loss: 0.20265647768974304, Type-Color-Style Accuracy: 0.9722222089767456
Usage Loss: 0.0005722008645534515, Usage Accuracy: 1.0
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step
Accuracy: 0.8617283950617284
Precision: 0.6618003489533782
Recall: 0.7462344337972899
F1 Score: 0.6894096165550821
Classification Report (Type-Color-Style):
              precision    recall  f1-score   support

         0.0       0.95      0.89      0.92      4389
         1.0       0.37      0.60      0.46       471

    accuracy                           0.86      4860
   macro avg       0.66      0.75      0.69      4860
weighted avg       0.90      0.86      0.88      4860

Usage Accuracy: 1.0
Usage Precision: 1.0
Usage Recall: 1.0
Usage F1 Score: 1.0
Classification Report (Usage):
                precision

In [24]:
# if you dont want to train other time
# use one of the saved models to see the output
# model = load_model("multi_label_model.keras") 

image_path = "source_directory\\Images\\A20419.jpg" # A20419 AL25303

thresholds = {
    "TYPE": 0.5,
    "COLOR": 0.1,
    "STYLE": 0.1,
    "USAGE": 0.5
}
color_mapping = {
    "Black": (0, 0, 0),          
    "Blue": (0, 0, 255),         
    "Brown": (139, 69, 19),      
    "Dark Beige": (190, 152, 112),  
    "Dark Grey": (169, 169, 169),   
    "Green": (0, 255, 0),        
    "Light Beige": (226, 193, 150), 
    "Light Grey": (211, 211, 211),  
    "Metallic": (153, 153, 153), 
    "Multicolor": (255, 255, 255), 
    "Orange": (255, 165, 0),     
    "Pink": (255, 192, 203),     
    "Purple": (128, 0, 128),     
    "Red": (255, 0, 0),         
    "White": (255, 255, 255),    
    "Yellow": (255, 255, 0)      
}

type_color_style_pred, usage_pred, type_color_style_binary, usage_binary = predict_single_image(model, image_path, thresholds)
decoded_labels = decode_predictions_with_probabilities(type_color_style_pred, usage_pred, thresholds)
predicted_colors = predict_colors(image_path, color_mapping, num_colors=3)

print("Decoded Predictions:", decoded_labels)
print("Matched Colors:", predicted_colors)

1/1 ━━━━━━━━━━━━━━━━━━━━ 38s 38s/step
Decoded Predictions: {'USAGE': [('wallcovering', 0.9999985694885254)], 'TYPE': [('wallpaper', 0.985772967338562)], 'STYLE': [('illustrative', 0.2087855488061905), ('stripes; checks; and zigzags', 0.19310256838798523), ('ethnic & oriental', 0.138838529586792), ('plain', 0.1199197992682457), ('textured', 0.10479345172643661), ('geometric', 0.1000237688422203)], 'COLOR': [('white', 0.4667302072048187), ('yellow', 0.4499638080596924), ('metallic', 0.3013284504413605), ('purple', 0.16771180927753448), ('light beige', 0.14465388655662537), ('orange', 0.11642416566610336)]}
Matched Colors: ['Light Grey', 'Dark Grey', 'Metallic']
